In [1]:
import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt

def build_model(hp, input_shape):
    model = keras.Sequential()
    model.add(keras.layers.Input(shape=input_shape))
    
    for i in range(hp.Int('num_layers', 1, 3)):
        model.add(keras.layers.Dense(
            units=hp.Int('units_' + str(i), min_value=16, max_value=128, step=16),
            activation='relu'))
        if hp.Boolean('dropout'):
            model.add(keras.layers.Dropout(rate=0.5))

    model.add(keras.layers.Dense(1))  # Output layer for regression

    optimizer_choice = hp.Choice('optimizer', ['adam', 'sgd', 'rmsprop'])
    if optimizer_choice == 'adam':
        optimizer = keras.optimizers.Adam(
            learning_rate=hp.Choice('adam_learning_rate', [1e-4, 1e-3, 1e-2, 1e-1]))
    elif optimizer_choice == 'sgd':
        optimizer = keras.optimizers.SGD(
            learning_rate=hp.Choice('sgd_learning_rate', [1e-4, 1e-3, 1e-2, 1e-1]))
    elif optimizer_choice == 'rmsprop':
        optimizer = keras.optimizers.RMSprop(
            learning_rate=hp.Choice('rmsprop_learning_rate', [1e-4, 1e-3, 1e-2, 1e-1]))
    
    model.compile(optimizer=optimizer,
                  loss='mean_squared_error',
                  metrics=['mean_absolute_percentage_error'])
    
    return model

# Example usage
input_shape = (10,)  # Example input shape

# Create the Keras Tuner RandomSearch object
tuner = kt.RandomSearch(
    hypermodel=lambda hp: build_model(hp, input_shape),
    objective='val_mean_absolute_percentage_error',
    max_trials=10,
    executions_per_trial=3,  # Execute each trial 3 times
    directory='my_dir',
    project_name='intro_to_kt')

# Generate sample data
X_train = tf.random.normal((1000, 10))
y_train = tf.random.normal((1000,))

# Search for the best hyperparameters
tuner.search(X_train, y_train, epochs=50, validation_split=0.2)

# Get the best model
best_model = tuner.get_best_models(num_models=1)[0]

# Print a summary of the best model
best_model.summary()



Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
1                 |1                 |num_layers
48                |48                |units_0
True              |True              |dropout
rmsprop           |rmsprop           |optimizer
0.001             |0.001             |adam_learning_rate


Epoch 1/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 4s 41ms/step - loss: 1.4817 - mean_absolute_percentage_error: 240.6546 - val_loss: 0.9967 - val_mean_absolute_percentage_error: 409.0336
Epoch 2/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 1.5022 - mean_absolute_percentage_error: 263.0779 - val_loss: 0.9879 - val_mean_absolute_percentage_error: 392.8690
Epoch 3/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 1.5171 - mean_absolute_percentage_error: 226.5293 - val_loss: 0.9794 - val_mean_absolute_percentage_error: 380.3427
Epoch 4/50
25/25 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - loss: 1.5909 - mean_absolute_percentage_error: 255.4974 - val_loss: 0.9703 - val_mean_absolute_p

KeyboardInterrupt: 